In [1]:
import torch
from torch import nn
from torch.utils.data import dataset as Dataset
from torch.utils.data import dataloader as DataLoader
import pandas as pd
import numpy as np
import os

from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
test = %pwd
data = pd.read_csv(f'{test}/kaggle1_data/train.csv')
# print(data.duplicated().sum()) #=0
newdf = data.interpolate()
diff = newdf['Age'].compare(data['Age'])
diff.index

Index([  5,  17,  19,  26,  28,  29,  31,  32,  36,  42,
       ...
       832, 837, 839, 846, 849, 859, 863, 868, 878, 888],
      dtype='int64', length=177)

In [3]:
###计算熵增益 H(Survived|Pclass) = -p1log2p1-p2log2p2
#####################耻辱柱：条件不要加df[]，会变成df&df报错！！！
# survive_cls1 = newdf[(newdf['Pclass']==1) & (newdf['Survived']==1)]
# cls1 = newdf[newdf['Pclass']==1]

lenth = len(newdf)
pct = [[] for i in range(2)] #0.63, 0.47, 0.24   H(S|1)  H(S|2)  H(S|3)
for i in range(1, 4):
    all_mem = len(newdf[newdf['Pclass']==i])
    survive = len(newdf[(newdf['Pclass']==i) & (newdf['Survived']==1)])
    sur_pct = round(survive/all_mem, 2) #活下来的占该cls的比例
    mem_pct = round(all_mem/lenth, 2) #该class占总的比例
    pct[0].append(sur_pct)
    pct[1].append(mem_pct)

HSx_1 = round(-np.log2(pct[0][0]) * pct[0][0] - np.log2(1 - pct[0][0]) * (1 - pct[0][0]), 2) #0.95
HSx_2 = round(-np.log2(pct[0][1]) * pct[0][1] - np.log2(1 - pct[0][1]) * (1 - pct[0][1]), 2) #1
HSx_3 = round(-np.log2(pct[0][2]) * pct[0][2] - np.log2(1 - pct[0][2]) * (1 - pct[0][2]), 2) #0.8
HSx = pct[1][0] * HSx_1 + pct[1][1] * HSx_2 + pct[1][2] * HSx_3 #0.878
HS = round(-np.log2(0.3)*0.3 * 3, 2) #1.56
HS_gain = HS - HSx #熵增益
HS_gain

0.682

In [4]:
newdf.loc[newdf['Sex']=='male', 'Sex'] = 0
newdf.loc[newdf['Sex']=='female', 'Sex'] = 1

In [5]:
def check_HS_gain(col, df):
    assert type(col) == str
    typ = list(df.groupby(col).groups.keys())
    num = range(len(typ))
    HS_x = []
    sur_pc = []
    for j in num:
        ind = typ[j]
        condition1 = df['Survived'] == 1
        condition2 = df[col] == ind
        intersection = df[(condition1) & (condition2)]
        sur_pct = round(len(intersection) / len(df[condition2]), 2) #注意condition2是索引不是df，默认长度等于df的
        all_pct = round(len(df[condition2]) / len(df), 2) #这里也是
        hs_xj = -np.log2(sur_pct) * sur_pct - np.log2(1 - sur_pct) * (1 - sur_pct)
        hs_part = hs_xj * all_pct
        HS_x.append(hs_part)
        sur_pc.append([sur_pct, all_pct, hs_xj, ind, condition2, len(df)])
    uniform = round(1 / len(typ), 2)
    HS_h0 = -np.log2(uniform) #后面*uniform*num等于乘以1了就不写了
    HS_observe = sum(HS_x)
    return HS_h0 - HS_observe

In [6]:
check_HS_gain('Pclass', newdf)

0.7245742809106628

In [7]:
numeric_idx = newdf.dtypes[newdf.dtypes != 'object'].index
label_set = torch.tensor(newdf[numeric_idx]['Survived'], dtype=torch.float32)
newdf_d = newdf[numeric_idx].drop(['Parch', 'PassengerId', 'Survived'], axis=1)
newdf_d = newdf_d.apply(lambda x: (x-x.mean())/x.std())
# newdf_d.plot(kind='scatter', x='Survived', y='Fare')
# plt.show()
newdf_d1 = pd.get_dummies(newdf_d, columns=['Pclass', 'SibSp']).astype(float)
# newdf_d1
dataset = newdf_d1.values
dataset.shape

(891, 12)

In [8]:
class Data_test1(Dataset.Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __getitem__(self, index):
        return torch.Tensor(self.data[index]), torch.Tensor(self.label[index])

    def __len__(self):
        return len(self.data)

In [9]:
pct = 0.6
num_train = round(len(dataset) * 0.6)
train_set, train_label = dataset[:num_train], label_set[:num_train]
test_set, test_label = dataset[num_train:], label_set[num_train:]

In [10]:
train_data = Data_test1(train_set, train_label)
test_data = Data_test1(test_set, test_label)

In [11]:
train_iter = DataLoader.DataLoader(dataset=train_data, batch_size=100, shuffle=True)
test_iter = DataLoader.DataLoader(dataset=test_data, batch_size=100, shuffle=True)

In [12]:
loss = nn.CrossEntropyLoss()
class test_net(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential()
        self.net.add_module('linear1', nn.Linear(12, 24))
        self.net.add_module('relu1', nn.ReLU())
        self.net.add_module('linear2', nn.Linear(24, 10))
        self.net.add_module('relu2', nn.ReLU())
        self.net.add_module('dense', nn.Linear(10, 2))
    def forward(self, X):
        for net in self.net:
            X = net(X)
        return X

In [21]:
def train(net, num_epochs, train_iter, loss, lr):
    if isinstance(net, nn.Module):
        net.train()
    train_ls = []
    metrics = Accumulator(3)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    for epoch in range(num_epochs):
        for X, y in train_iter:
            y = y.type(torch.long)
            # y = y.reshape((-1, 1))
            # print(X.shape, y.shape)
            optimizer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
            metrics.add(float(l * y.numel()), accuracy(net(X), y)[0], y.numel())
        if epoch % 50 == 0:
            train_ls.append([metrics[0] / metrics[2] , metrics[1] / metrics[2], accuracy(net(X), y)[1]]) 
    return train_ls

In [30]:
def accuracy(y_hat, y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    ##自制auc曲线看一下 TPP和FPP不是除以所有！是除以True P!   True F!
    auc = []
    counttp = 0
    countfp = 0
    for i in range(y.numel()):
        if y[i] - y_hat.type(y.dtype)[i] == -1:
            countfp += 1
        if (y[i] - y_hat.type(y.dtype)[i] == 0) and (y[i] == 1):
            counttp += 1
    auc.append([counttp / y[y==1].sum(), countfp / y[y==0].sum()])
    return float(cmp.type(y_hat.dtype).sum()), auc

In [15]:
class Accumulator:
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, index):
        return self.data[index]


In [16]:
def evaluate_accuracy(net, data_iter):
    if isinstance(net, nn.Module):
        net.eval()
    metric = Accumulator(2)
    for X, y in data_iter:
        metric.add(accuracy(net(X), y)[0], y.numel())
    return metric[0] / metric[1]

In [17]:
# for i, (data, label) in enumerate(train_iter):
#     if i==2:
#         break
#     print(net(data), label.reshape((-1, 1)))

In [31]:
net = test_net()
# test_X = torch.ones((2, 12))
# net.parameters()
train(net, 1000, train_iter, loss, 0.03)
# net

[[0.6545580641131534, 0.6598130841121496, [[tensor(0.5556), tensor(inf)]]],
 [0.5701261342827907, 0.7087044163459777, [[tensor(0.3125), tensor(inf)]]],
 [0.547565238813561, 0.7200703247894883, [[tensor(0.7333), tensor(inf)]]],
 [0.5339787217541748, 0.7264467413504982, [[tensor(0.5625), tensor(inf)]]],
 [0.5227701298522168, 0.7326451852885107, [[tensor(0.7895), tensor(inf)]]],
 [0.5152678839629594, 0.7365751945489072, [[tensor(0.6250), tensor(inf)]]],
 [0.5078608767856825, 0.7404042599434906, [[tensor(0.6364), tensor(inf)]]],
 [0.5033900100011638, 0.7430518944537636, [[tensor(0.4667), tensor(inf)]]],
 [0.49878131653880414, 0.7449926585405645, [[tensor(0.7273), tensor(inf)]]],
 [0.49594783129218667, 0.7469051122116999, [[tensor(0.6364), tensor(inf)]]],
 [0.493261951956967, 0.7484768780196616, [[tensor(0.6667), tensor(inf)]]],
 [0.4905405004148614, 0.7496616177892362, [[tensor(0.6000), tensor(inf)]]],
 [0.48867020688633905, 0.7510317694807719, [[tensor(0.8000), tensor(inf)]]],
 [0.4864030

In [35]:
a = torch.empty(3, dtype=torch.long).random_(3)
b = torch.randn((3, 3)).argmax(axis=1)
a, b, b.dtype

(tensor([1, 0, 0]), tensor([0, 1, 0]), torch.int64)

In [32]:
a = torch.tensor([[1, 0], [0, 1], [1, 0]])
b = torch.tensor([1, 0, 1])
a[a.argmax(axis=1)==0].sum()

tensor(2)